# Using Models as Layers in Another Model

In this notebook, we show how you can use Keras models as Layers within a larger model and still perform pruning on that model.

In [ ]:
# Import required packages

import tensorflow as tf
import mann

from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# Load the data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

# Convert images from grayscale to RGB
x_train = tf.image.grayscale_to_rgb(tf.Variable(x_train.reshape(-1, 28, 28, 1)))
x_test = tf.image.grayscale_to_rgb(tf.Variable(x_test.reshape(-1, 28, 28, 1)))

## Model Creation

In the following cells, we create two models and put them together to create a larger model. The first model, called the `preprocess_model`, takes in images, divides the pixel values by 255 to ensure all values are between 0 and 1, resized the image to a height and width of 40 pixels. It then performs training data augmentation by randomly flips some images across the y-axis, randomly rotates images, and randomly translates the images.

The second model, called the `true_model`, contains the logic for performing prediction on images. It contains blocks of convolutional layers followed by max pooling and dropout layers. The output of these blocks is flattened and passed through fully-connected layers to output predicted class probabilities.

These two models are combined in the `training_model` to be trained.

In [ ]:
preprocess_model = tf.keras.models.Sequential()
preprocess_model.add(tf.keras.layers.Rescaling(1./255))
preprocess_model.add(tf.keras.layers.Resizing(40, 40, input_shape = (None, None, 3)))
preprocess_model.add(tf.keras.layers.RandomFlip('horizontal'))
preprocess_model.add(tf.keras.layers.RandomRotation(0.1))
preprocess_model.add(tf.keras.layers.RandomTranslation(0.1, 0.1))

true_model = tf.keras.models.Sequential()
true_model.add(mann.layers.MaskedConv2D(16, padding = 'same', input_shape = (40, 40, 3)))
true_model.add(mann.layers.MaskedConv2D(16, padding = 'same'))
true_model.add(tf.keras.layers.MaxPool2D())
true_model.add(tf.keras.layers.Dropout(0.2))
true_model.add(mann.layers.MaskedConv2D(32, padding = 'same', activation = 'relu'))
true_model.add(mann.layers.MaskedConv2D(32, padding = 'same', activation = 'relu'))
true_model.add(tf.keras.layers.MaxPool2D())
true_model.add(tf.keras.layers.Dropout(0.2))
true_model.add(mann.layers.MaskedConv2D(64, padding = 'same', activation = 'relu'))
true_model.add(mann.layers.MaskedConv2D(64, padding = 'same', activation = 'relu'))
true_model.add(tf.keras.layers.MaxPool2D())
true_model.add(tf.keras.layers.Dropout(0.2))
true_model.add(tf.keras.layers.Flatten())
true_model.add(mann.layers.MaskedDense(256, activation = 'relu'))
true_model.add(mann.layers.MaskedDense(256, activation = 'relu'))
true_model.add(mann.layers.MaskedDense(10, activation = 'softmax'))

training_input = tf.keras.layers.Input((None, None, 3))
training_x = preprocess_model(training_input)
training_output = true_model(training_x)
training_model = tf.keras.models.Model(
    training_input,
    training_output
)

training_model.compile(
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy'],
    optimizer = 'adam'
)

training_model.summary()

## Model Training

In this cell, we create the `ActiveSparsification` object to continually sparsify the model as it trains, and train the model.

In [ ]:
callback = mann.utils.ActiveSparsification(
    0.80,
    sparsification_rate = 5
)

training_model.fit(
    x_train,
    y_train,
    epochs = 200,
    batch_size = 512,
    validation_split = 0.2,
    callbacks = [callback]
)

## Convert the model to not have masking layers

In the following cell, we configure the model to remove masking layers and replace them with non-masking native TensorFlow layers. We then perform prediction on the resulting model and present the results.

In [ ]:
model = mann.utils.remove_layer_masks(training_model)
preds = model.predict(x_test).argmax(axis = 1)
print(confusion_matrix(y_test, preds))
print(classification_report(y_test, preds))

## Save only the model that performs prediction

Lastly, save only the part of the model that performs prediction

In [ ]:
model.layers[2].save('ModelLayer.h5')